<a href="https://colab.research.google.com/github/harnalashok/LLMs/blob/main/chatDoctor_bioGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<iframe src="https://www.kaggle.com/embed/aliabdin1/llm-01-how-to-use-llms-with-hugging-face?cellIds=4&kernelSessionId=140351055" height="300" style="margin: 0 auto; width: 100%; max-width: 950px;" frameborder="0" scrolling="auto" title="⚡LLM 01 - How to use LLMs with Hugging Face⚡"></iframe>

<iframe src="https://www.kaggle.com/embed/aliabdin1/llm-01-how-to-use-llms-with-hugging-face?cellIds=4&kernelSessionId=140351055" height="300" style="margin: 0 auto; width: 100%; max-width: 950px;" frameborder="0" scrolling="auto" title="⚡LLM 01 - How to use LLMs with Hugging Face⚡"></iframe>

In [ ]:
# Last amended: 18th April, 2024
# Ref: https://huggingface.co/Narrativaai/BioGPT-Large-finetuned-chatdoctor
# HuggingFace Notebooks: https://huggingface.co/docs/transformers/notebooks

## Objective(s)

To demonstrate how chatdoctor works.    
Medical models on huggingface are [here](https://huggingface.co/models?other=medical&sort=likes)


## Install packages & call libraries
This colab notebook has secret tokens saved.

In [1]:
# 0.1 Sacremoses is for the translation model Helsinki-NLP/opus-mt-en-es:

!pip install sacremoses==0.0.53

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895240 sha256=6d9cb2c57117e2606b96cf3b8282fefdc9220618b9fce39286f3f41a8b69787f
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [2]:
# 0.2 Install datasets and transformers:
#     About transformers, see below

! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [3]:
# 0.3 What is accelerate? See below

!pip install -U accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.1 MB/s eta 0:00:00


In [4]:
# 0.4 Call libraries:

from transformers import pipeline
import pandas as pd

In [5]:
# 0.5 Current present work directory:
! pwd

# 0.6 Make folder to store datasets/models
! mkdir cache

/content


## Expt

In [6]:
# Ref: https://huggingface.co/Narrativaai/BioGPT-Large-finetuned-chatdoctor
# 1.0
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [7]:
# 1.1
model_id = "Narrativaai/BioGPT-Large-finetuned-chatdoctor"

# 1.2
tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")

# 1.3
model = AutoModelForCausalLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/566k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.28G [00:00<?, ?B/s]

In [8]:
# 1.4
model = model.to("cuda")

In [9]:
# 2.0
def answer_question(prompt, temperature=0.1, top_p=0.75, top_k=40, num_beams=2, **kwargs,):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")
    generation_config = GenerationConfig(
                                          temperature=temperature,
                                          top_p=top_p,
                                          top_k=top_k,
                                          num_beams=num_beams,
                                          **kwargs,
                                        )

    with torch.no_grad():
        generation_output = model.generate(
                                            input_ids=input_ids,
                                            attention_mask=attention_mask,
                                            generation_config=generation_config,
                                            return_dict_in_generate=True,
                                            output_scores=True,
                                            max_new_tokens=512,
                                            eos_token_id=tokenizer.eos_token_id

                                        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s,
                              skip_special_tokens=True
                              )

    return output.split(" Response:")[1]




In [10]:
example_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.

### Instruction:
If you are a doctor, please answer the medical questions based on the patient's description.

### Input:
Hi i have sore lumps under the skin on my legs. they started on my left ankle and are approx 1 - 2cm diameter and are spreading up onto my thies. I am eating panadol night and anti allergy pills (Atarax). I have had this for about two weeks now. Please advise.

### Response:
"""

print(answer_question(example_prompt))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

 Hi, Dear thanks for the query to Chat Doctor virtual clinic. I studied your query in full details updated from you. I understood your health concerns. Based on your query data, In my opinion, Dear You seem to suffer from- -Chronic Urticaria with Urticaria Plantar. Treatment-Consult ER Dermatologist who would treat it accordingly. Hope this reply would help you to evaluate your case and treat it with your doctors in time to come. Hope this would resolve your query and worry and Anxiety accompanied by it. Welcome for any further query in this regard to ME. I would love to help you out.Awaiting any further query. Wish you fast recovery from this intriguing health problem. Have a Good Day. Chat Doctor. N.M.S. Genl-CVTS -Senior Surgical Consultant


In [ ]:
######### DONE ##############